# Simulated Probit

In [ ]:
import numpy as np
import gurobipy as grb

In [ ]:
seed = 777
nbDraws = 10
U_y = np.array([0.4, 0.5, 0.2, 0.3, 0.1, 0])
nbY = len(U_y)

In [ ]:
rho = 0.5
Covar = rho * np.ones((nbY, nbY)) + (1 - rho) * np.eye(nbY)

In [ ]:
E = np.linalg.eigh(Covar)
V = E[0]
Q = E[1]
SqrtCovar = Q.dot(np.diag(np.sqrt(V))).dot(Q.T)

In [ ]:
epsilon_iy = np.random.normal(0,1,nbDraws*nbY).reshape(nbDraws,nbY).dot(SqrtCovar)
u_iy = epsilon_iy + U_y

In [ ]:
ui = np.max(u_iy, axis=1)
s_y = np.sum((u_iy.T - ui).T == 0, axis=0) / nbDraws

In [ ]:
opt_assign = [(i, j) for i in range(nbY) for j in range(nbDraws)]

m = grb.Model()
vars = m.addVars(opt_assign, obj=epsilon_iy.T.ravel(), name='vars')
m.ModelSense = -1
m.addConstrs(vars.sum('*', i) == 1/nbDraws for i in range(nbDraws))
m.addConstrs(vars.sum(i, '*') == s_y[i] for i in range(nbY))

In [ ]:
m.optimize()
if m.Status == grb.GRB.Status.OPTIMAL:
    pi = m.getAttr('pi')
    Uhat_y = -np.subtract(pi[nbDraws:nbY+nbDraws], pi[nbY + nbDraws - 1])
    print('U_y (true and recovered)')
    print(U_y)
    print(Uhat_y)